## <b> ■ 복습</b>
    1장. 강화학습 개념
        강화 + 머신러닝 ( 데이터를 통해서 스스로 학습하는 알고리즘 ) : 신경망
            보상을 통해서 인공신경망을 학습시키는 것
    
    2장. 순차적 문제들의 세상(환경)을 해결하려면 세상에 대한 수학적 정의가 필요한데 수학적 도구 2가지
        1. MDP
        2. 벨만 방정식
    
    3장. 다이나믹 프로그래밍
        1. 정책 이터레이션 (벨만 기대방정식)
        2. 가치 이터레이션 (벨만 최적방정식)
        
        에이전트에게 만들어준 환경을 에이전트가 잘 살아갈 수 있도록 계산해주는 것
        
        차원이 복잡해지고 차원수가 늘어나면 계산이 오래걸리므로 학습이 필요
        
    4장. 강화학습
        1. SARSA : 시간차 학습방법으로 바로바로 학습하는 강화학습 방법
            On policy : 벨만 기대 방정식
        2. QLearning : SARSA처럼 바로바로 학습을 하는데 행동과 학습이 분리된 방법
            Off policy : 벨만 최적 방정식
        
        장애물이 움직이는 환경이면 SARSA와 QLearning으로는 해결할 수 없다. 그래서 인공 신경망이 필요
        
    5장. DeepSARSA (SARSA + ANN)
        Q_TABLE을 인공 신경망으로 근사하는 방법을 사용해서 에이전트가 환경에 스스로 적응해 나가도록 구현
        
        DeepSARSA 코드 구현
        

#### 문제3. 딥살사 클래스 함수중 train_model 함수의 tf.GradientTape() 가 무엇인가요? 
```python
def train_model(self, state, action, reward, next_state, next_action, done):
    if self.epsilon > self.epsilon_min:
        self.epsilon *= self.epsilon_decay

    # 학습 파라메터
    model_params = self.model.trainable_variables
    with tf.GradientTape() as tape:
        tape.watch(model_params)
        predict = self.model(state)[0]
        one_hot_action = tf.one_hot([action], self.action_size)
        predict = tf.reduce_sum(one_hot_action * predict, axis=1)

        # done = True 일 경우 에피소드가 끝나서 다음 상태가 없음

        next_q = self.model(next_state)[0][next_action]
        target = reward + (1 - done) * self.discount_factor * next_q

        # MSE 오류 함수 계산
        loss = tf.reduce_mean(tf.square(target - predict))

    # 오류함수를 줄이는 방향으로 모델 업데이트
    grads = tape.gradient(loss, model_params)

    self.optimizer.apply_gradients(zip(grads, model_params))
```

    Tensorflow 2의 tape 기능을 이용하면 신경망 학습 중에 발생하는 기울기 값을 확인할 수 있다.
    기울기 값을 뽑을 수 있게 되면 그래프로 시각화 할 수 도 있으며 기울기가 점점 작아지는 것을 확인함으로써
    학습이 잘되고 있는지 Local Minima에 빠지지 않았는지 등을 확인할 수 있어 유용하다.
    
    날코딩했던 때는 기울기를 직접 뽑을 수 있었다. 

In [ ]:
for i in range(10):
    x_batch = x  # 어차피 1장이기때문에 x 만 넣는다.
    t_batch = t[:1]
    grad = net.numerical_gradient(x_batch, t_batch)

    for key in ('W1', 'W2', 'b1', 'b2'):
        net.params[key] -= 0.01 * grad[key]
    #acc = net.accuracy(x_batch, t_batch)
    y = net.predict(x_batch)
    y_hat = np.argmax(y, axis=1)
    print(y_hat)

    케라스로 넘어오면서 아래와 같이 기울기는 케라스 내장함수에 돌고 model.fit 만 하면 되었다.
    그래서 기울기를 확인할 수 있는 길이 없어졌다. 

```python
import  tensorflow  as   tf
tf.random.set_seed(777)  # 시드를 설정한다.

import  numpy  as  np
from  tensorflow.keras.models  import  Sequential   # 신경망 모델 구성
from  tensorflow.keras.layers  import  Dense  # 완전 연결계층 
from  tensorflow.keras.optimizers  import   SGD  # 경사감소법 
from  tensorflow.keras.losses   import   mse    #  오차함수 

# 데이터 준비
x = np.array( [ [0, 0], [1, 0], [0, 1], [1, 1] ] )
y = np.array( [ [0], [0], [0], [1] ] )

#모델 구성하기 
model = Sequential()

#단층 퍼셉트론 구현하기
model.add( Dense( 1, input_shape =( 2,  ), activation ='linear')  ) 

# 모델 준비하기
model.compile( optimizer= SGD(), 
                     loss= mse, 
                     metrics = ['acc'] ) # list 형태로 평가지표를 전달한다.  

# 학습 시키기 
model.fit(x, y, epochs = 500) 
```

In [ ]:
import tensorflow as tf

x = tf.ones((2, 2))
print(x)

'''
tf.Tensor(
[[1. 1.]
 [1. 1.]], shape=(2, 2), dtype=float32)
'''

with tf.GradientTape() as t:
    t.watch(x) # tape에 기록
    y = tf.reduce_sum(x)
print(y)

# tf.Tensor(4.0, shape=(), dtype=float32)

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x) # y=4x
    z = tf.multiply(y, y) # z = y^2

print(z)

# tf.Tensor(16.0, shape=(), dtype=float32)

dz_dx = t.gradient(z, x) # z를 x에 대해 미분
print(dz_dx)
'''
tf.Tensor(
[[8. 8.]
 [8. 8.]], shape=(2, 2), dtype=float32)
'''

    위의 설명 사이트의 코드를  신경망으로 코렙에서 수행할 수 있도록 정리

In [ ]:
import tensorflow as tf
import numpy as np

mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train.min(), x_train.max()

# 차원을 1 늘려줍니다
x_train = tf.expand_dims(x_train, -1)
x_test = tf.expand_dims(x_test, -1)

x_train.shape, x_test.shape


# Dataset을 만듭니다
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)


#STEP 1. Model을 정의합니다.

# layer 정의
input_ = tf.keras.layers.Input(shape=(28, 28, 1))
x = tf.keras.layers.Conv2D(32, 3, activation='relu')(input_)
x = tf.keras.layers.Conv2D(64, 3, activation='relu')(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
output_ = tf.keras.layers.Dense(10, activation='softmax')(x)



# model을 정의
model = tf.keras.models.Model(input_, output_)
model.summary()


#STEP 2. Loss Function을 정의합니다.
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()

#STEP 3. Optimizer를 정의합니다.
optimizer = tf.keras.optimizers.Adam()

# STEP 4. Metric을 정의합니다.
train_loss = tf.keras.metrics.Mean()
train_acc = tf.keras.metrics.SparseCategoricalAccuracy()
test_loss = tf.keras.metrics.Mean()
test_acc = tf.keras.metrics.SparseCategoricalAccuracy()


# STEP 5. Train/Test step 함수를 정의합니다.

'''
GradientTape (그라디언트 테이프)
텐서플로는 자동 미분(주어진 입력 변수에 대한 연산의 그래디언트(gradient)를 계산하는 것) 을 위한 tf.GradientTape API를 제공합니다.

tf.GradientTape는 컨텍스트(context) 안에서 실행된 모든 연산을 테이프(tape)에 "기록"합니다.

그 다음 텐서플로는 후진 방식 자동 미분(reverse mode differentiation)을 사용해 테이프에 "기록된" 연산의 그래디언트를 계산합니다.
'''


@tf.function
def train_step(images, labels):
    # 미분을 위한 GradientTape을 적용합니다.
    with tf.GradientTape() as tape:
        # 1. 예측 (prediction)
        predictions = model(images)
        # 2. Loss 계산
        loss = loss_function(labels, predictions)
    
    # 3. 그라디언트(gradients) 계산
    gradients = tape.gradient(loss, model.trainable_variables)
    
    # 4. 오차역전파(Backpropagation) - weight 업데이트
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    # loss와 accuracy를 업데이트 합니다.
    train_loss(loss)
    train_acc(labels, predictions)
    
@tf.function
def test_step(images, labels):
    # 1. 예측 (prediction)
    predictions = model(images)
    # 2. Loss 계산
    loss = loss_function(labels, predictions)
    
    # Test셋에 대해서는 gradient를 계산 및 backpropagation 하지 않습니다.
    
    # loss와 accuracy를 업데이트 합니다.
    test_loss(loss)
    test_acc(labels, predictions)
    
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
        train_step(images, labels)
        
    for test_images, test_labels in test_ds:
        test_step(test_images, test_labels)

    template = '에포크: {}, 손실: {:.5f}, 정확도: {:.2f}%, 테스트 손실: {:.5f}, 테스트 정확도: {:.2f}%'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_acc.result()*100,
                           test_loss.result(),
                           test_acc.result()*100))


### 문제. 아래의 3x3의 1로 채워진 x행렬을 만들고 아래의 z함수를 x에 대해서 미분한 미분계수(기울기)를 출력하시오
    y = 9x
    z = y^3

In [ ]:
x = tf.ones(3, 3)

with tf.GradientTape() as t:
    t.watch(x)
    y = tf.reduce_sum(x) 
    z = tf.multiply(y, y, y) 

dz_dx = t.gradient(z, x) 
print(dz_dx)

####  문제4. train_model 함수 안에 self.model.trainable_variables 안의 내용이 무엇인가요?

In [ ]:
def train_model(self, state, action, reward, next_state, next_action, done):
    if self.epsilon > self.epsilon_min:
        self.epsilon *= self.epsilon_decay

    # 학습 파라메터
    model_params = self.model.trainable_variables
    print(model_params)

#### 문제5. 상태를 인공신경망에 넣었을때 출력되는 예측값은 무엇인가요 ?

In [ ]:
with tf.GradientTape() as tape:
    tape.watch(model_params)
    predict = self.model(state)[0]
    print(predict)

#### 문제6. one_hot_action 에서 출력되는 결과는 무엇인가요?

In [ ]:
with tf.GradientTape() as tape:
    tape.watch(model_params)
    predict = self.model(state)[0]

    one_hot_action = tf.one_hot([action], self.action_size)
    print(one_hot_action)

#### 문제7. 최종 predict 에서 출력되는 결과는 무엇인가요 ? 

In [ ]:
def train_model(self, state, action, reward, next_state, next_action, done):
    if self.epsilon > self.epsilon_min:
        self.epsilon *= self.epsilon_decay

    # 학습 파라메터
    model_params = self.model.trainable_variables

    with tf.GradientTape() as tape:
        tape.watch(model_params)
        predict = self.model(state)[0]

        one_hot_action = tf.one_hot([action], self.action_size)

        predict = tf.reduce_sum(one_hot_action * predict, axis=1)
        print(predict)

#### 문제8. 출력되는 결과를 확인하세요 !